# Main Results - Dispatch

Developed by Siobhan Powell, 2022. 


In [1]:
import os
os.chdir('..')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import scipy
import os.path
import pandas
import matplotlib
import shutil
import copy

from simple_dispatch import generatorData
from simple_dispatch import bidStack
from simple_dispatch import dispatch
from simple_dispatch import generatorDataShort
from future_grid import FutureDemand
from future_grid import FutureGrid
from simple_dispatch import StorageModel

Load generator model:

In [3]:
gd_short = pickle.load(open('IntermediateOutputs/generator_data_short_WECC_2019.obj', 'rb'))

# Dispatch for penlevel = 1.0

In [4]:
penlevel = 1.0
solar = 3.5
wind = 3

fuel = 1
ev_scenario_date = '20220313'
max_penlevel = 0.1
timer_names = {'':'Timers9pm', '_midnighttimer':'Timers12am', '_NoTimers':'TimersNone', '_RandomTimers':'TimersRandom'}
max_rate = 10000 # assumes average 4 hour storage 
cap = 40000

for ev_scenario in ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']: # Access scenarios


    for i, ev_timers in enumerate(['_RandomTimers', '', '_midnighttimer', '_NoTimers']):
        # Case without storage before
        ev_workplace_bool=False
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, ev_timers=ev_timers, ev_pen=penlevel, 
                             ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, ev_scenario_date=ev_scenario_date, 
                             weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/Fuel1_Solar35_Wind3/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)
        print(save_str)
        grid.check_overgeneration(save_str=save_str)
        grid.run_dispatch(0.1, save_str, result_date='20220408') 

        # Case with 10GW storage before
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                             ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, 
                             ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/Fuel1_Solar35_Wind3/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)+'_storagebefore'
        print(save_str)
        grid.check_overgeneration(save_str=save_str, change_demand=False)
        grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
        grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
        grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
        grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
        grid.run_dispatch(0.1, save_str, result_date='20220408') 

    # workplace control - repeat above without timers but with workplace controls
    ev_workplace_bool=True
    ev_timers = '_NoTimers'
    for wp_control in ['avgem', 'minpeak']:
        # Case without storage
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/Fuel1_Solar35_Wind3/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
        print(save_str)
        grid.check_overgeneration(save_str=save_str)
        grid.run_dispatch(0.1, save_str, result_date='20220408')

        # Case with storage before
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)

        save_str = 'Results/Fuel1_Solar35_Wind3/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
        print(save_str)
        grid.check_overgeneration(save_str=save_str, change_demand=False)
        grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
        grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
        grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
        grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
        grid.run_dispatch(0.1, save_str, result_date='20220408') 

        
ev_scenario = 'BusinessAsUsual'
ev_workplace_bool=True
ev_timers=''
wp_control = 'minpeak'
grid = FutureGrid(gd_short)
grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                            ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                            ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
save_str = 'Results/Fuel1_Solar35_Wind3/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
print(save_str)
grid.check_overgeneration(save_str=save_str)
grid.run_dispatch(0.1, save_str, result_date='20220408')                

# storage before
grid = FutureGrid(gd_short)
grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                            ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                            ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
save_str = 'Results/Fuel1_Solar35_Wind3/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
print(save_str)
grid.check_overgeneration(save_str=save_str, change_demand=False)
grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
grid.run_dispatch(0.1, save_str, result_date='20220408')  


Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_UniversalHome_TimersRandom_noWPcontrol_penlevel1.0
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 14673
Storage Capacity:  58693
Results

3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 18193
Storage Capacity:  72773
Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_UniversalHome_TimersNone_WPcontrol_avgem_penlevel1.0_storagebefore
Solving optimization.
----Capacity too low----
T

23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 8686
Storage Capacity:  34745
Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_HighHome_Timers12am_noWPcontrol_penlevel1.0_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Com

42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 7424
Storage Capacity:  29699
Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_LowHome_HighWork_TimersRandom_noWPcontrol_penlevel1.0_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete

59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 7923
Storage Capacity:  31695
Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_LowHome_HighWork_TimersNone_WPcontrol_avgem_penlevel1.0_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Comple

75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 7694
Storage Capacity:  30778
Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_LowHome_LowWork_Timers12am_noWPcontrol_penlevel1.0_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71

90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 10897
Storage Capacity:  43591
Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_BusinessAsUsual_TimersMixed_WPcontrol_minpeak_penlevel1.0_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Com

# Run dispatch for penlevel = 0.5

In [5]:
penlevel = 0.5
solar = 3.5
wind = 3

fuel = 1
ev_scenario_date = '20220313'
max_penlevel = 0.1
timer_names = {'':'Timers9pm', '_midnighttimer':'Timers12am', '_NoTimers':'TimersNone', '_RandomTimers':'TimersRandom'}
max_rate = 10000 # assumes average 4 hour storage 
cap = 40000

for ev_scenario in ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']: # Access scenarios


    for i, ev_timers in enumerate(['_RandomTimers', '', '_midnighttimer', '_NoTimers']):
        # Case without storage before
        ev_workplace_bool=False
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, ev_timers=ev_timers, ev_pen=penlevel, 
                             ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, ev_scenario_date=ev_scenario_date, 
                             weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/Fuel1_Solar35_Wind3/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)
        print(save_str)
        grid.check_overgeneration(save_str=save_str)
        grid.run_dispatch(0.1, save_str, result_date='20220408') 

        # Case with 10GW storage before
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                             ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, 
                             ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/Fuel1_Solar35_Wind3/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)+'_storagebefore'
        print(save_str)
        grid.check_overgeneration(save_str=save_str, change_demand=False)
        grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
        grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
        grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
        grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
        grid.run_dispatch(0.1, save_str, result_date='20220408') 

    # workplace control - repeat above without timers but with workplace controls
    ev_workplace_bool=True
    ev_timers = '_NoTimers'
    for wp_control in ['avgem', 'minpeak']:
        # Case without storage
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/Fuel1_Solar35_Wind3/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
        print(save_str)
        grid.check_overgeneration(save_str=save_str)
        grid.run_dispatch(0.1, save_str, result_date='20220408')

        # Case with storage before
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)

        save_str = 'Results/Fuel1_Solar35_Wind3/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
        print(save_str)
        grid.check_overgeneration(save_str=save_str, change_demand=False)
        grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
        grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
        grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
        grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
        grid.run_dispatch(0.1, save_str, result_date='20220408') 

        
ev_scenario = 'BusinessAsUsual'
ev_workplace_bool=True
ev_timers=''
wp_control = 'minpeak'
grid = FutureGrid(gd_short)
grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                            ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                            ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
save_str = 'Results/Fuel1_Solar35_Wind3/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
print(save_str)
grid.check_overgeneration(save_str=save_str)
grid.run_dispatch(0.1, save_str, result_date='20220408')                

# storage before
grid = FutureGrid(gd_short)
grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                            ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                            ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
save_str = 'Results/Fuel1_Solar35_Wind3/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
print(save_str)
grid.check_overgeneration(save_str=save_str, change_demand=False)
grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
grid.run_dispatch(0.1, save_str, result_date='20220408')  


Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_UniversalHome_TimersRandom_noWPcontrol_penlevel0.5
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 5544
Storage Capacity:  22179
Results/

3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 7126
Storage Capacity:  28506
Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_UniversalHome_TimersNone_WPcontrol_avgem_penlevel0.5_storagebefore
Solving optimization.
----Capacity too low----
Tr

26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 4537
Storage Capacity:  18150
Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_HighHome_Timers12am_noWPcontrol_penlevel0.5_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Com

50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 3906
Storage Capacity:  15627
Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_LowHome_HighWork_TimersRandom_noWPcontrol_penlevel0.5_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete

67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 4156
Storage Capacity:  16625
Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_LowHome_HighWork_TimersNone_WPcontrol_avgem_penlevel0.5_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Comple

82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 4041
Storage Capacity:  16167
Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_LowHome_LowWork_Timers12am_noWPcontrol_penlevel0.5_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78

98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 5447
Storage Capacity:  21789
Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_BusinessAsUsual_TimersMixed_WPcontrol_minpeak_penlevel0.5_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Comp

# Run dispatch for case with no EVs

In [6]:
    # Use FutureGrid to run scenarios without EVs
grid = FutureGrid(gd_short)
grid.set_up_scenario(year=2035, solar=3.5, wind=3, fuel=1, ev_workplace_bool=False, evs_bool=False)
save_str = 'Results/NoEVs_year2035_solar'+str(3.5)+'x_wind'+str(3)+'x'
grid.check_overgeneration(save_str=save_str)
grid.run_dispatch(1.0, save_str, result_date='20220408') 

1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
Error!
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete


In [5]:
max_rate = 10000 # assumes average 4 hour storage 
cap = 40000

grid = FutureGrid(gd_short)
grid.set_up_scenario(year=2035, solar=3.5, wind=3, fuel=1, ev_workplace_bool=False, evs_bool=False)
save_str = 'Results/NoEVs_year2035_solar'+str(3.5)+'x_wind'+str(3)+'x_storagebefore'
grid.check_overgeneration(save_str=save_str, change_demand=False)
grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
grid.run_dispatch(0.1, save_str, result_date='20220408')  

Solving optimization.
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete


# Run Dispatch for other EV Penetration Levels

In [7]:
solar = 3.5
wind = 3

fuel = 1
ev_scenario_date = '20220313'
max_penlevel = 0.05
timer_names = {'':'Timers9pm', '_midnighttimer':'Timers12am', '_NoTimers':'TimersNone', '_RandomTimers':'TimersRandom'}
max_rate = 10000 # assumes average 4 hour storage 
cap = 40000


for penlevel in [0.1, 0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9, 1.0]: #penlevel = 0.5

    for ev_scenario in ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']: # Access scenarios


        for i, ev_timers in enumerate(['_RandomTimers', '', '_midnighttimer', '_NoTimers']):
            # Case without storage before
            ev_workplace_bool=False
            grid = FutureGrid(gd_short)
            grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, ev_timers=ev_timers, ev_pen=penlevel, 
                                 ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, ev_scenario_date=ev_scenario_date, 
                                 weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
            save_str = 'Results/Fuel1_Solar35_Wind3_Curves/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)
            print(save_str)
            grid.check_overgeneration(save_str=save_str)
            grid.future.demand.to_csv(save_str+'_demanddf_beforestorage_beforedispatch_20220408.csv')
            grid.run_dispatch(max_penlevel, save_str, result_date='20220408') 

            # Case with 10GW storage before
            grid = FutureGrid(gd_short)
            grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                 ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, 
                                 ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
            save_str = 'Results/Fuel1_Solar35_Wind3_Curves/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)+'_storagebefore'
            print(save_str)
            grid.check_overgeneration(save_str=save_str, change_demand=False)
            grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
            grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
            grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
            grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
#             grid.run_dispatch(0.1, save_str, result_date='20220408') 
            grid.future.demand.to_csv(save_str+'_futuredemand_df_20220408.csv')

        # workplace control - repeat above without timers but with workplace controls
        ev_workplace_bool=True
        ev_timers = '_NoTimers'
        for wp_control in ['avgem', 'minpeak']:
            # Case without storage
            grid = FutureGrid(gd_short)
            grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                    ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                    ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
            save_str = 'Results/Fuel1_Solar35_Wind3_Curves/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
            print(save_str)
            grid.check_overgeneration(save_str=save_str)
            grid.future.demand.to_csv(save_str+'_demanddf_beforestorage_beforedispatch_20220408.csv')
            grid.run_dispatch(max_penlevel, save_str, result_date='20220408')

            # Case with storage before
            grid = FutureGrid(gd_short)
            grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                    ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                    ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)

            save_str = 'Results/Fuel1_Solar35_Wind3_Curves/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
            print(save_str)
            grid.check_overgeneration(save_str=save_str, change_demand=False)
            grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
            grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
            grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
            grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
#             grid.run_dispatch(0.1, save_str, result_date='20220408') 
            grid.future.demand.to_csv(save_str+'_futuredemand_df_20220408.csv')


    ev_scenario = 'BusinessAsUsual'
    ev_workplace_bool=True
    ev_timers=''
    wp_control = 'minpeak'
    grid = FutureGrid(gd_short)
    grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
    save_str = 'Results/Fuel1_Solar35_Wind3_Curves/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
    print(save_str)
    grid.check_overgeneration(save_str=save_str)
    grid.future.demand.to_csv(save_str+'_demanddf_beforestorage_beforedispatch_20220408.csv')
    grid.run_dispatch(max_penl evel, save_str, result_date='20220408')                

    # storage before
    grid = FutureGrid(gd_short)
    grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
    save_str = 'Results/Fuel1_Solar35_Wind3_Curves/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
    print(save_str)
    grid.check_overgeneration(save_str=save_str, change_demand=False)
    grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
    grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
    grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
    grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
#     grid.run_dispatch(0.1, save_str, result_date='20220408')  
    grid.future.demand.to_csv(save_str+'_futuredemand_df_20220408.csv')


Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_UniversalHome_TimersRandom_noWPcontrol_penlevel0.1
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 1405
Storage Capacity:  5620
Re

13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 1219
Storage Capacity:  4878
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_HighHome_Timers9pm_noWPcontrol_penlevel0.1_storagebefore
Solving optimization.
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_HighHome_Timers12am_noWPcontrol_penlevel0.1
--

46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 1092
Storage Capacity:  4371
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_LowHome_HighWork_Timers12am_noWPcontrol_penlevel0.1_storagebefore
Solving optimization.
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_LowHome_HighWork_TimersNone_noWPcontrol_penlevel0.1
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% C

69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 1186
Storage Capacity:  4744
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_LowHome_LowWork_TimersNone_noWPcontrol_penlevel0.1_storagebefore
Solving optimization.
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_LowHome_LowWork_TimersNone_WPcontrol_avgem_penlevel0.1
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0%

94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 2878
Storage Capacity:  11515
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_UniversalHome_TimersNone_noWPcontrol_penlevel0.2_storagebefore
Solving optimization.
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_UniversalHome_TimersNone_WPcontrol_avgem_penlevel0.2
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Co

...

3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 6963
Storage Capacity:  27853
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_LowHome_LowWork_Timers9pm_noWPcontrol_penlevel0.9_storagebefore
Solving optimization.
Results/Fuel1_Solar35_W

26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 24540
Storage Capacity:  98161
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_UniversalHome_Timers9pm_noWPcontrol_penlevel1.0_storagebefore
Solving optimization.
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_UniversalHome_Timers12am_noWPcontrol_penlevel1.0
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Compl

55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 8686
Storage Capacity:  34745
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_HighHome_Timers12am_noWPcontrol_penlevel1.0_storagebefore
Solving optimization.
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_HighHome_TimersNone_noWPcontrol_penlevel1.0
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% C

84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 8072
Storage Capacity:  32289
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_LowHome_HighWork_TimersNone_noWPcontrol_penlevel1.0_storagebefore
Solving optimization.
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_LowHome_HighWork_TimersNone_WPcontrol_avgem_penlevel1.0
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65

Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_LowHome_LowWork_TimersNone_WPcontrol_avgem_penlevel1.0_storagebefore
Solving optimization.
Results/Fuel1_Solar35_Wind3_Curves/fuel1_solar3.5_wind3_LowHome_LowWork_TimersNone_WPcontrol_minpeak_penlevel1.0
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Comple

# Dispatch for Case with High Renewables and 50% EVs

In [5]:
penlevel = 0.5
solar = 5
wind = 5

fuel = 1
ev_scenario_date = '20220313'
max_penlevel = 0.1
timer_names = {'':'Timers9pm', '_midnighttimer':'Timers12am', '_NoTimers':'TimersNone', '_RandomTimers':'TimersRandom'}
max_rate = 10000 # assumes average 4 hour storage 
cap = 40000

for ev_scenario in ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']: # Access scenarios


    for i, ev_timers in enumerate(['_RandomTimers', '', '_midnighttimer', '_NoTimers']):
        # Case without storage before
        ev_workplace_bool=False
#         grid = FutureGrid(gd_short)
#         grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, ev_timers=ev_timers, ev_pen=penlevel, 
#                              ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, ev_scenario_date=ev_scenario_date, 
#                              weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
#         save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)
#         print(save_str)
#         grid.check_overgeneration(save_str=save_str)
#         grid.run_dispatch(0.1, save_str, result_date='20220408') 

        # Case with 10GW storage before
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                             ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, 
                             ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)+'_storagebefore'
        print(save_str)
        grid.check_overgeneration(save_str=save_str, change_demand=False)
        grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
        grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
        grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
        grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
        grid.run_dispatch(0.1, save_str, result_date='20220408') 

    # workplace control - repeat above without timers but with workplace controls
    ev_workplace_bool=True
    ev_timers = '_NoTimers'
    for wp_control in ['avgem', 'minpeak']:
        # Case without storage
#         grid = FutureGrid(gd_short)
#         grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
#                                                 ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
#                                                 ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
#         save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
#         print(save_str)
#         grid.check_overgeneration(save_str=save_str)
#         grid.run_dispatch(0.1, save_str, result_date='20220408')

        # Case with storage before
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)

        save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
        print(save_str)
        grid.check_overgeneration(save_str=save_str, change_demand=False)
        grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
        grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
        grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
        grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
        grid.run_dispatch(0.1, save_str, result_date='20220408') 

        
ev_scenario = 'BusinessAsUsual'
ev_workplace_bool=True
ev_timers=''
wp_control = 'minpeak'
# grid = FutureGrid(gd_short)
# grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
#                                             ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
#                                             ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
# save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
# print(save_str)
# grid.check_overgeneration(save_str=save_str)
# grid.run_dispatch(0.1, save_str, result_date='20220408')                

# storage before
grid = FutureGrid(gd_short)
grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                            ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                            ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
print(save_str)
grid.check_overgeneration(save_str=save_str, change_demand=False)
grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
grid.run_dispatch(0.1, save_str, result_date='20220408')  


Results/Fuel1_Solar5_Wind5_2035/fuel1_solar5_wind5_UniversalHome_TimersRandom_noWPcontrol_penlevel0.5_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 

Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 0
Storage Capacity:  0
Results/Fuel1_Solar5_Wind5_2035/fuel1_solar5_wind5_HighHome_TimersNone_noWPcontrol_penlevel0.

----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 0
Storage Capacity:  0
Results/Fuel1_Solar5_Wind5_2035/fuel1_solar5_wind5_LowHome_HighWork_TimersNone_WPcontrol_minpeak_penlevel0.5_storag

----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 0
Storage Capacity:  0


# Dispatch for case with Low Renewables and 50% EVs

In [6]:
penlevel = 0.5
solar = 2.5
wind = 2.5

fuel = 1
ev_scenario_date = '20220313'
max_penlevel = 0.1
timer_names = {'':'Timers9pm', '_midnighttimer':'Timers12am', '_NoTimers':'TimersNone', '_RandomTimers':'TimersRandom'}
max_rate = 10000 # assumes average 4 hour storage 
cap = 40000

for ev_scenario in ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']: # Access scenarios


    for i, ev_timers in enumerate(['_RandomTimers', '', '_midnighttimer', '_NoTimers']):
        # Case without storage before
        ev_workplace_bool=False
#         grid = FutureGrid(gd_short)
#         grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, ev_timers=ev_timers, ev_pen=penlevel, 
#                              ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, ev_scenario_date=ev_scenario_date, 
#                              weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
#         save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)
#         print(save_str)
#         grid.check_overgeneration(save_str=save_str)
#         grid.run_dispatch(0.1, save_str, result_date='20220408') 

        # Case with 10GW storage before
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                             ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, 
                             ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/Fuel1_Solar25_Wind25_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)+'_storagebefore'
        print(save_str)
        grid.check_overgeneration(save_str=save_str, change_demand=False)
        grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
        grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
        grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
        grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
        grid.run_dispatch(0.1, save_str, result_date='20220408') 

    # workplace control - repeat above without timers but with workplace controls
    ev_workplace_bool=True
    ev_timers = '_NoTimers'
    for wp_control in ['avgem', 'minpeak']:
        # Case without storage
#         grid = FutureGrid(gd_short)
#         grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
#                                                 ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
#                                                 ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
#         save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
#         print(save_str)
#         grid.check_overgeneration(save_str=save_str)
#         grid.run_dispatch(0.1, save_str, result_date='20220408')

        # Case with storage before
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)

        save_str = 'Results/Fuel1_Solar25_Wind25_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
        print(save_str)
        grid.check_overgeneration(save_str=save_str, change_demand=False)
        grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
        grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
        grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
        grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
        grid.run_dispatch(0.1, save_str, result_date='20220408') 

        
ev_scenario = 'BusinessAsUsual'
ev_workplace_bool=True
ev_timers=''
wp_control = 'minpeak'
# grid = FutureGrid(gd_short)
# grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
#                                             ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
#                                             ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
# save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
# print(save_str)
# grid.check_overgeneration(save_str=save_str)
# grid.run_dispatch(0.1, save_str, result_date='20220408')                

# storage before
grid = FutureGrid(gd_short)
grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                            ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                            ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
save_str = 'Results/Fuel1_Solar25_Wind25_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
print(save_str)
grid.check_overgeneration(save_str=save_str, change_demand=False)
grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
grid.run_dispatch(0.1, save_str, result_date='20220408')  


Results/Fuel1_Solar25_Wind25_2035/fuel1_solar2.5_wind2.5_UniversalHome_TimersRandom_noWPcontrol_penlevel0.5_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Re

Results/Fuel1_Solar25_Wind25_2035/fuel1_solar2.5_wind2.5_HighHome_Timers12am_noWPcontrol_penlevel0.5_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 0

Results/Fuel1_Solar25_Wind25_2035/fuel1_solar2.5_wind2.5_LowHome_HighWork_TimersNone_WPcontrol_avgem_penlevel0.5_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Ra

Storage Rate Result: 0
Storage Capacity:  0
Results/Fuel1_Solar25_Wind25_2035/fuel1_solar2.5_wind2.5_BusinessAsUsual_TimersMixed_WPcontrol_minpeak_penlevel0.5_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
10

In [7]:
tmp = pd.read_csv('Results/Fuel1_Solar35_Wind3/fuel1_solar3.5_wind3_LowHome_LowWork_Timers9pm_noWPcontrol_penlevel0.5_overgeneration.csv')

In [11]:
pd.to_datetime(tmp['datetime']).dt.date.unique()

array([datetime.date(2019, 3, 17), datetime.date(2019, 3, 22),
       datetime.date(2019, 3, 23), datetime.date(2019, 3, 24),
       datetime.date(2019, 3, 25), datetime.date(2019, 3, 28),
       datetime.date(2019, 3, 30), datetime.date(2019, 3, 31),
       datetime.date(2019, 4, 6), datetime.date(2019, 4, 7),
       datetime.date(2019, 4, 9), datetime.date(2019, 4, 10),
       datetime.date(2019, 4, 11), datetime.date(2019, 4, 13),
       datetime.date(2019, 4, 14), datetime.date(2019, 4, 17),
       datetime.date(2019, 4, 18), datetime.date(2019, 4, 20),
       datetime.date(2019, 4, 21), datetime.date(2019, 4, 23),
       datetime.date(2019, 4, 26), datetime.date(2019, 4, 27),
       datetime.date(2019, 4, 28), datetime.date(2019, 4, 30),
       datetime.date(2019, 5, 1), datetime.date(2019, 5, 2),
       datetime.date(2019, 5, 4), datetime.date(2019, 5, 5),
       datetime.date(2019, 5, 11), datetime.date(2019, 5, 12),
       datetime.date(2019, 5, 13), datetime.date(2019, 5, 17),

# Dispatch for other renewable cases with no EVs

In [12]:
    # Use FutureGrid to run scenarios without EVs
grid = FutureGrid(gd_short)
grid.set_up_scenario(year=2035, solar=5, wind=5, fuel=1, ev_workplace_bool=False, evs_bool=False)
save_str = 'Results/NoEVs_year2035_solar'+str(5)+'x_wind'+str(5)+'x'
grid.check_overgeneration(save_str=save_str)
grid.run_dispatch(1.0, save_str, result_date='20220408') 

grid = FutureGrid(gd_short)
grid.set_up_scenario(year=2035, solar=2.5, wind=2.5, fuel=1, ev_workplace_bool=False, evs_bool=False)
save_str = 'Results/NoEVs_year2035_solar'+str(2.5)+'x_wind'+str(2.5)+'x'
grid.check_overgeneration(save_str=save_str)
grid.run_dispatch(1.0, save_str, result_date='20220408') 

1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8

# Dispatch for case with High Renewables with 100% EVs

In [5]:
penlevel = 1.0
solar = 5
wind = 5

fuel = 1
ev_scenario_date = '20220313'
max_penlevel = 0.1
timer_names = {'':'Timers9pm', '_midnighttimer':'Timers12am', '_NoTimers':'TimersNone', '_RandomTimers':'TimersRandom'}
max_rate = 10000 # assumes average 4 hour storage 
cap = 40000

for ev_scenario in ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']: # Access scenarios


    for i, ev_timers in enumerate(['_RandomTimers', '', '_midnighttimer', '_NoTimers']):
        # Case without storage before
        ev_workplace_bool=False
#         grid = FutureGrid(gd_short)
#         grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, ev_timers=ev_timers, ev_pen=penlevel, 
#                              ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, ev_scenario_date=ev_scenario_date, 
#                              weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
#         save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)
#         print(save_str)
#         grid.check_overgeneration(save_str=save_str)
#         grid.run_dispatch(0.1, save_str, result_date='20220408') 

        # Case with 10GW storage before
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                             ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, 
                             ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)+'_storagebefore'
        print(save_str)
        grid.check_overgeneration(save_str=save_str, change_demand=False)
        grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
        grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
        grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
        grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
        grid.run_dispatch(0.1, save_str, result_date='20220408') 

    # workplace control - repeat above without timers but with workplace controls
    ev_workplace_bool=True
    ev_timers = '_NoTimers'
    for wp_control in ['avgem', 'minpeak']:
        # Case without storage
#         grid = FutureGrid(gd_short)
#         grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
#                                                 ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
#                                                 ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
#         save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
#         print(save_str)
#         grid.check_overgeneration(save_str=save_str)
#         grid.run_dispatch(0.1, save_str, result_date='20220408')

        # Case with storage before
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)

        save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
        print(save_str)
        grid.check_overgeneration(save_str=save_str, change_demand=False)
        grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
        grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
        grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
        grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
        grid.run_dispatch(0.1, save_str, result_date='20220408') 

        
ev_scenario = 'BusinessAsUsual'
ev_workplace_bool=True
ev_timers=''
wp_control = 'minpeak'
# grid = FutureGrid(gd_short)
# grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
#                                             ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
#                                             ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
# save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
# print(save_str)
# grid.check_overgeneration(save_str=save_str)
# grid.run_dispatch(0.1, save_str, result_date='20220408')                

# storage before
grid = FutureGrid(gd_short)
grid.set_up_scenario(year=2035, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                            ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                            ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
save_str = 'Results/Fuel1_Solar5_Wind5_2035/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
print(save_str)
grid.check_overgeneration(save_str=save_str, change_demand=False)
grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220408'+'.csv')
grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
grid.run_dispatch(0.1, save_str, result_date='20220408')  


Results/Fuel1_Solar5_Wind5_2035/fuel1_solar5_wind5_UniversalHome_TimersRandom_noWPcontrol_penlevel1.0_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 

Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 0
Storage Capacity:  0
Results/Fuel1_Solar5_Wind5_2035/fuel1_solar5_wind5_HighHome_TimersNone_noWPcontrol_penlevel1.

----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 0
Storage Capacity:  0
Results/Fuel1_Solar5_Wind5_2035/fuel1_solar5_wind5_LowHome_HighWork_TimersNone_WPcontrol_minpeak_penlevel1.0_storag

----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 0
Storage Capacity:  0
